# Model Formulations and Solving Them

We will work through several of the model formulations and also solve them using Gurobi. Once you have the formulation, coding it and sending it to Gurobi is the "easy" part. See below for the most common steps to creating and solving a model when working with Gurobi.

Every basic Gurobi optimization model can be constructed with the code template shown below.  This template is provided as a starting point for each problem formulations below.  Feel free to use it for your work.

```python
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''

''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit', 7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
m.update()

''' Create objective function and update model '''
m.setObjective()
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
m.update()

''' Optimize model '''
m.optimize()

''' Print results '''

```

## Capital Budgeting

In reality, many corporate projects are multi-year projects that require investment for multiple years. Suppose we had the following possible projects over a four-year time horizon with the costs and NPVs in millions of US dollars. Additionally, there are some restrictions as described below.

| Project | Year 1 Capital | Year 2 Capital | Year 3 Capital | Year 4 Capital | NPV |
| -- | :--: | :--: | :--: | :--: |  :--: |
| A | 3 | 0 | 0 | 0 | 2 |
| B | 0 | 5 | 1 | 0 | 3 |
| C | 1 | 2 | 1 | 2 | 1 |
| D | 10 | 4 | 2 | 0 | 5 |
| E | 2 | 0 | 5 | 1 | 4 | 

**Policy Restrictions:**
- No more than 4 total projects can be undertaken
- Projects D and E are mutually exclusive; if one is undertaken, the other cannot be
- Projects A and C must be undertaken together, if at all
- Project D is dependent on Project B; Project D cannot be undertaken unless Project B is undertaken

**Capital Available:**
- Year 1 = \$12 million
- Year 2 = \$8 million
- Year 3 = \$8 million
- Year 4 = \$4 million

### Formulation

| **Data** | |
| -- | -- |
| $I$ | Set of possible projects, {A, B, C, D, E}, indexed by $i$ |
| $J$ | Set of years indexed by $j$ |
| $c_{ij}$ | Capital investment for project $i$ during year $j$ |
| $r_{i}$ | NPV of project $i$ |
| $b_{j}$ | Budget for year $j$ |


$$\text{Let } x_{i} = \begin{cases} 1 & \text{if project } i \text{ is selected} \\ 
    0 & \text{otherwise} \end{cases}$$

$$\max \sum_{i} r_{i}x_{i}$$ 

$$\text{s.t.} \sum_{i} c_{ij}x_{i} \leq b_{j} \quad \forall \quad j \quad \text{yearly budget}$$

$$ \sum_{i} x_{i} \leq 4 \quad \text{No more than 4 total projects}$$
$$x_{D} + x_{E} \leq 1 \quad \text{Projects D and E are mutually exclusive} $$
$$x_{A} = x_{C} \quad \text{Projects A and C must be undertaken together if at all}$$
$$x_{D} \leq x_{B} \quad \text{Project D is dependent on Project B}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
projects = ['A', 'B', 'C', 'D', 'E']
years = ['Year1', 'Year2', 'Year3', 'Year4']
c = [[3, 0, 0, 0],
     [0, 5, 1, 0],
     [1, 2, 1, 2],
     [10, 4, 2, 0],
     [2, 0, 5, 1]]

r = [2, 3, 1, 5, 4]
b = [12, 8, 8, 4]

''' Create Gurobi model object '''
m = gp.Model('project_selection2') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [m.addVar(vtype=GRB.BINARY, name=f'x_{projects[i]}') for i in range(len(projects))]
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum(r[i]*x[i] for i in range(len(projects))))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# Add the yearly budget constraints
for j in range(len(years)):
    m.addConstr(gp.quicksum(c[i][j]*x[i] for i in range(len(projects))) <= b[j],
                name=f'budget_{years[j]}')

# No more than 4 projects
m.addConstr(gp.quicksum(x[i] for i in range(len(projects))) <= 4,
            name='no_more_than_4')

# Projects D and E are mutually exclusive
m.addConstr(m.getVarByName('x_D') + m.getVarByName('x_E') <= 1,
            name='D_and_E_mutually_exclusive')

# Projects A and C must be undertaken together if at all
m.addConstr(m.getVarByName('x_A') == m.getVarByName('x_C'),
            name='A_and_C_together')

# Project D is dependent on Project B
m.addConstr(m.getVarByName('x_D') <= m.getVarByName('x_B'),
            name='D_depends_on_B')

m.update()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal NPV from investing in projects is ${m.objVal} millions of US dollars')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Producing Products

Your firm produces three products. Each product must go through three different departments before it is complete, taking various amounts of time for each step. Every product has an associated profit for each unit sold but also has a maximum demand for each month. Each department has limited labor available each month. It also costs to set up the machines associated with each product. What is the best production plan for next month?

| | Product 1 | Product 2 | Product 3 |
| -- | :--: | :--: | :--: |
| Profit / Unit | \$25 | \$28 | \$30 |
| Dept A labor hours | 1.5 | 3 | 2 | 
| Dept B labor hours | 2 | 1 | 2.5 |
| Dept C labor hours | 0.25 | 0.25 | 0.25 |
| Maximum Production | 175 | 150 | 140 |

Labor Hours Available in Each Department:
- Dept A = 450
- Dept B = 350
- Dept C = 50

Setup Costs:
- Product 1 = \$400
- Product 2 = \$550
- Product 3 = \$600

### Formulation Attempt 1
| Data | | 
| -- | -- |
| $I$ | set of products being produced indexed by $i$ |
| $J$ | set of departments products must go through indexed by $j$ |
| $d_{i}$ | maximum monthly demand for product $i$ |
| $p_{i}$ | profit per unit for product $i$ |
| $t_{ij}$ | time needed for product $i$ in department $j$ |
| $T_{j}$ | monthly labor hours available for department $j$ |
| $c_{i}$ | setup cost for product $i$ |

$$\text{Let } x_{i} = \text{ number of product } i \text{ to produce next month}$$
$$\max \sum_{i} p_{i} x_{i} - \sum_{i}c_{i}$$
$$ \text{s.t.} $$
$$ \sum_{i} t_{ij}x_{i} \leq T_{j} \quad \forall \quad j \quad \text{labor hours for each department}$$ 
$$ 0 \leq x_{i} \leq d_{i} \quad \forall \quad i \quad \text{ bounds }$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
products = ['Product1', 'Product2', 'Product3']
depts = ['DeptA', 'DeptB', 'DeptC']
p = [25, 28, 30]
d = [175, 150, 140]
s = [400, 550, 600]
t = [[1.5, 2, 0.25], [3, 1, 0.25], [2, 2.5, 0.25]]
T = [450, 350, 50]

range_products = range(len(products))
range_depts = range(len(depts))


# Suppress output
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

''' Create Gurobi model object '''
m = gp.Model('production', env=env) # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)


''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [m.addVar(name=f'{products[i]}', lb=0.0, ub=d[i]) for i in range_products]
m.update()

''' Create objective function and update model '''
m.setObjective(gp.quicksum(p[i]*x[i] - s[i] for i in range_products))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# labor constraint for each dept
m.addConstrs((gp.quicksum(t[i][j]*x[i] for i in range_products) <= T[j] for j in range_depts),
            name='labor')

m.update()

# m.display()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'Total cost for production is: ${m.objVal:,.2f}')
for v in m.getVars():
    print(f'{v.varName}: {v.X}')

### Formulation Attempt 2

How can we incorporate the setup costs into the model more intelligently? Right now they are simply subtracted out of the profit regardless of if we produce any of a particular product. Note, that we are producing all three products in the solution we found. Could you find a higher (better) profit if you take the setup costs into account more holistically?

| Data | | 
| -- | -- |
| $I$ | set of products being produced indexed by $i$ |
| $J$ | set of departments products must go through indexed by $j$ |
| $d_{i}$ | maximum monthly demand for product $i$ |
| $p_{i}$ | profit per unit for product $i$ |
| $t_{ij}$ | time needed for product $i$ in department $j$ |
| $T_{j}$ | monthly labor hours available for department $j$ |
| $c_{i}$ | setup cost for product $i$ |

$$\text{Let } x_{i} = \text{ number of product } i \text{ to produce next month}$$
$$y_{i} = \begin{cases} 1 & \text{if machines are set up for proudct } i \\ 
    0 & \text{otherwise} \end{cases}$$
$$\max \sum_{i} p_{i} x_{i} - \sum_{i}c_{i}y_{i}$$
$$ \text{s.t.} $$
$$ \sum_{i} t_{ij}x_{i} \leq T_{j} \quad \forall \quad j \quad \text{labor hours for each department}$$ 
$$ x_{i} \leq d_{i}y_{i} \quad \forall \quad i \quad \text{ maximum production if machines are set up }$$
$$ x_{i} \geq 0 \quad \text{non-negativity}$$ 

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
products = ['Product1', 'Product2', 'Product3']
depts = ['DeptA', 'DeptB', 'DeptC']
p = [25, 28, 30]
d = [175, 150, 140]
c = [400, 550, 600]
t = [[1.5, 2, 0.25], [3, 1, 0.25], [2, 2.5, 0.25]]
T = [450, 350, 50]

range_products = range(len(products))
range_depts = range(len(depts))


# Suppress output
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

''' Create Gurobi model object '''
m = gp.Model('production', env=env) # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [m.addVar(name=f'{products[i]}', lb=0.0) for i in range_products]
# Add the binary variables here
y = [m.addVar(vtype=GRB.BINARY, name=f'setup_for_{products[i]}') for i in range_products]
m.update()

''' Create objective function and update model '''
# fix the obj fn
m.setObjective(gp.quicksum(p[i]*x[i] - c[i]*y[i] for i in range_products))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# labor constraint for each dept
m.addConstrs((gp.quicksum(t[i][j]*x[i] for i in range_products) <= T[j] for j in range_depts),
            name='labor')

# max production based on setting up machines
for i in range_products:
    m.addConstr(x[i] <= d[i]*y[i], name=f'max_production_{products[i]}')

m.update()

''' Optimize model '''
m.optimize()

''' Print results '''
print(f'Total profit for production is: ${m.objVal:,.2f}')
for v in m.getVars():
    print(f'{v.varName}: {v.X}')

## Locating Emergency Vehicles

A city needs to decide where to locate sites for their emergency vehicles. They have seven potential locations. An important requirement is that regardless of where the sites are located, all nine districts of the city must be reachable for at least one of the emergency sites within three minutes. Where should the sites be located?

### Formulation

| Data | | 
| -- | -- |
| $L$ | set of locations indexed by $i$ |
| $D$ | set of districts needed to be covered indexed by $j$ |
| $c_{ij}$ | indicator data (1 or 0) if site $i$ can reach district $j$ in required time |

$$\text{Let} \quad y_{i} = \begin{cases} 1 & \text{if site } i \text{ is chosen} \\ 
    0 & \text{otherwise} \end{cases}$$

$$\min \sum_{i} y_{i}$$
$$\text{s.t.} \quad \quad \quad \quad$$
$$\sum_{i} c_{ij} y_{i} \geq 1\quad \forall \quad j \quad \text{cover district } j$$

The important aspect of this formulation is the data $c_{ij}$ that indicates whether site $i$ covers district $j$. Let's see if we can pull in the data from a .csv file and use it.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

''' Import or define problem data '''
# Read in the data that indicates which districts are covered by sites
# Each row (i) is a potential site and each column (j) is a district
data = pd.read_csv('./data/ems.csv', index_col=0)
# data.info()

# convert the values of the DataFrame to 2-dimensional list
c = data.to_numpy().tolist()
# c

''' Create Gurobi model object '''
m = gp.Model('emergency_vehicles') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
y = []
for i in range(len(c)):
    y.append(m.addVar(vtype=GRB.BINARY, name=f'y_{i}'))

m.update()

# print(m.getVars())

''' Create objective function and update model '''
m.setObjective(gp.quicksum(y[i] for i in range(len(c))))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# make sure each district is covered by at least site
# Using a for loop to add each cover constraint one at a time
for j in range(len(c[0])):
    m.addConstr((gp.quicksum(c[i][j]*y[i] for i in range(len(c))) >= 1),
                 name=f'cover_district_{j}')

m.update()

m.display()

# ''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal number of sites chosen is: {m.objVal}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

## Assigning New Hires to Job Locations

You are the hiring manager at a consulting firm headquartered in New York. You've recently hired three new W&M graduates and must decide at which of the regional offices to place them. Each office can only have one new hire. What is the optimal assignment of personnel to offices?

| Hiree \ Office | Omaha | Charlotte | Cincinatti |
| -- | :--: | :--: | :--: |
| Smith | \$800 | \$1,100 | \$1,200 |
| Jones | \$500 | \$1,600 | \$1,300 |
| Torres | \$500 | \$1,000 | \$2,300 |

### Formulation

| Data | |
| -- | -- |
| $I$ | set of hirees that must be assigned to offices indexed by $i$ |
| $J$ | set of offices that want a new hire indexed by $j$ |
| $c_{ij}$ | cost of relocating person $i$ to location $j$ |

$$\text{Let } x_{ij} = \begin{cases} 1 & \text{if person } i \text{ is assigned to location } j \\ 
    0 & \text{otherwise} \end{cases}$$

$$\min \sum_{i} \sum_{j} c_{ij}x_{ij}$$ 

$$\text{s.t.}$$
$$\sum_{j} x_{ij} = 1 \quad \forall \quad i \quad \text{each person goes to only 1 location}$$
$$\sum_{i} x_{ij} = 1 \quad \forall \quad j \quad \text{each location only gets 1 person}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''
people = ['Smith', 'Jones', 'Torres']
locations = ['Omaha', 'Charlotte', 'Cincy']
c = [[800, 1100, 1200],
     [500, 1600, 1300],
     [500, 1000, 2300]]

range_p = range(len(people))
range_l = range(len(locations))

''' Create Gurobi model object '''
m = gp.Model('assignment') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MINIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit',7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
x = [[m.addVar(vtype=GRB.BINARY, name=f'send_{people[i]}_to_{locations[j]}') for j in range_l] for i in range_p]
m.update()
# m.getVars()

''' Create objective function and update model '''
m.setObjective(gp.quicksum(c[i][j]*x[i][j] for j in range_l for i in range_p))
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
# make sure each person only goes to one location
m.addConstrs((gp.quicksum(x[i][j] for j in range_l) == 1 for i in range_p), 
             name='each_person_to_one_location')

# make sure each location gets only one person
m.addConstrs((gp.quicksum(x[i][j] for i in range_p) == 1 for j in range_l),
             name='each_location_only_gets_1_person')


m.update()

m.display()

# # ''' Optimize model '''
m.optimize()

''' Print results '''
print(f'\nTotal cost of assignments is: ${m.objVal:,.2f}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.varName}: {v.X}')

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Professor of Business Analytics at William \& Mary.**